In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import sys
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import *
import bartz
import arviz as az

INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


In [3]:
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}
#proposal_probs = {"multi_grow": 0.5, "multi_prune": 0.5}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

bart = MultiBART(ndpost=50, nskip=20, n_trees=100, proposal_probs=proposal_probs, multi_tries=10)
bart.fit(X_train, y_train)

profiler.stop()
profiler.print()

Iterations: 100%|██████████| 70/70 [00:27<00:00,  2.58it/s]



  _     ._   __/__   _ _  _  _ _/_   Recorded: 13:05:11  Samples:  23795
 /_//_/// /_\ / //_// / //_'/ //     Duration: 27.153    CPU time: 26.922
/   _/                      v5.0.1

Profile at C:\Windows\Temp\ipykernel_9328\4126380201.py:4

27.152 <module>  C:\Windows\Temp\ipykernel_9328\4126380201.py:1
└─ 27.152 MultiBART.fit  bart_playground\bart.py:27
   └─ 27.151 MultiSampler.run  bart_playground\samplers.py:83
      └─ 27.076 MultiSampler.one_iter  bart_playground\samplers.py:338
         ├─ 23.140 MultiChange.propose  bart_playground\moves.py:33
         │  ├─ 9.677 MultiChange.try_propose  bart_playground\moves.py:344
         │  │  ├─ 5.613 Tree.change_split  bart_playground\params.py:391
         │  │  │  └─ 5.516 Tree.update_n  bart_playground\params.py:405
         │  │  │     ├─ 2.549 [self]  bart_playground\params.py
         │  │  │     ├─ 1.575 Tree.update_n  bart_playground\params.py:405
         │  │  │     │  ├─ 0.752 [self]  bart_playground\params.py
         │  │ 

In [5]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=50, nskip=20)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

default_proposal_probs = {"grow": 0.25, "prune": 0.25, "change": 0.4, "swap": 0.1}
bart_default = DefaultBART(ndpost=50, nskip=10, n_trees=100, proposal_probs=default_proposal_probs)
bart_default.fit(X_train, y_train)

INFO:2025-06-19 13:05:41,161:jax._src.xla_bridge:927: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-06-19 13:05:41,164:jax._src.xla_bridge:927: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
Iterations: 100%|██████████| 60/60 [00:01<00:00, 33.11it/s]


In [6]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz,
          "bart_default" : bart_default}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.022654767539919945,
 'rf': 0.022139023845392215,
 'lr': 0.048045521328019404,
 'btz': 0.022409798535525886,
 'bart_default': 0.022324879581491108}